# Autoencoders

Neste notebook, exploraremos os **Autoencoders**, uma classe fundamental de redes neurais utilizadas no aprendizado não supervisionado. O principal objetivo de um autoencoder é aprender uma representação eficiente (encoding) para um conjunto de dados, tipicamente para redução de dimensionalidade.

## Fundamentos

### O que é um Autoencoder?

Um autoencoder é uma rede neural projetada para reproduzir sua entrada na saída. Internamente, ele possui uma camada oculta que descreve um código usado para representar a entrada. A rede pode ser vista como consistindo de duas partes principais:

1.  **Encoder (Codificador)**: Uma função $f$ que comprime a entrada $x$ em uma representação latente $z$. $$z = f(x)$$
2.  **Decoder (Decodificador)**: Uma função $g$ que reconstrói a entrada $\hat{x}$ a partir da representação latente $z$. $$\hat{x} = g(z)$$

Se o autoencoder for capaz de aprender $g(f(x)) = x$ perfeitamente em todos os lugares, ele não é muito útil. Em vez disso, autoencoders são projetados para não serem capazes de copiar perfeitamente. O objetivo é restringir o modelo de forma que ele só consiga copiar aproximadamente, forçando-o a priorizar quais aspectos da entrada devem ser copiados. Isso geralmente é feito limitando a dimensão de $z$ (o gargalo ou *bottleneck*).

### Objetivo

O treinamento consiste em minimizar uma função de perda $L$, que mede a diferença entre a entrada original $x$ e a reconstrução $\hat{x}$. Uma escolha comum é o Erro Quadrático Médio (MSE):

$$ L(x, \hat{x}) = ||x - \hat{x}||^2 $$

### Breve Revisão de Redes Neurais

Para implementar nosso autoencoder, usaremos redes neurais densas (Fully Connected).

-   **Neurônio**: A unidade básica, que recebe entradas, aplica pesos ($W$), um viés ($b$) e uma função de ativação. A saída é $y = \text{ativação}(W \cdot x + b)$.
-   **Camadas**: Neurônios são organizados em camadas. Em uma camada densa, cada neurônio está conectado a todos os neurônios da camada anterior.
-   **Funções de Ativação**: Introduzem não-linearidade. Usaremos `ReLU` (Rectified Linear Unit) nas camadas ocultas e `Sigmoid` na saída (para escalar os pixels entre 0 e 1).

## Implementação com Keras e MNIST

### Biblioteca: Keras

O **Keras** é uma API de alto nível para redes neurais, escrita em Python e capaz de rodar sobre o TensorFlow. Ele foi desenvolvido com foco em facilitar a experimentação rápida.

Conceitos chave do Keras que usaremos:
-   **Input**: Define a forma da entrada.
-   **Dense**: Camada de rede neural densamente conectada.
-   **Model**: Agrupa camadas em um objeto com métodos de treinamento e inferência (`model.compile`, `model.fit`).

### Dataset: MNIST

Usaremos o dataset **MNIST**, que consiste em 70.000 imagens em tons de cinza de dígitos manuscritos (0-9), com dimensão 28x28 pixels.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

In [ ]:
# Carregar os dados (vamos usar apenas as imagens, pois é não supervisionado)
(x_train, _), (x_test, _) = mnist.load_data()

# Normalizar os dados para o intervalo [0, 1]
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Redimensionar as imagens de (28, 28) para vetores de tamanho 784
# Isso é necessário porque usaremos camadas Densas, que esperam vetores flat
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

print("Shape de treino:", x_train.shape)
print("Shape de teste:", x_test.shape)

In [ ]:
# Visualizar algumas amostras do dataset
n = 10  # Quantas imagens mostrar
plt.figure(figsize=(20, 2))
for i in range(n):
    ax = plt.subplot(1, n, i + 1)
    plt.imshow(x_train[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.suptitle("Amostras do MNIST", fontsize=16)
plt.show()

## Autoencoder Simples

Vamos criar um autoencoder básico com uma única camada oculta. A dimensão de entrada é 784 (28x28) e a dimensão latente (encoding) será 32. Isso significa um fator de compressão de 24.5x.

In [ ]:
# Dimensão da representação codificada
encoding_dim = 32

In [ ]:
# Input Placeholder
input_img = Input(shape=(784,))

In [ ]:
# Camada Encoder
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(encoding_dim, activation='relu', name="latent")(encoded)

In [ ]:
# Camada Decoder
decoded = Dense(128, activation='relu')(encoded)
decoded = Dense(784, activation='sigmoid')(decoded)

In [ ]:
# Modelo Autoencoder
autoencoder = Model(input_img, decoded)

# Modelo separado para o encoder
encoder = Model(input_img, encoded)

# Modelo separado para o decoder
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-2](encoded_input)
decoder_layer = autoencoder.layers[-1](decoder_layer)
decoder = Model(encoded_input, decoder_layer)

# Resumo da arquitetura
autoencoder.summary()

### Treinamento

Configuramos o modelo para minimizar a perda `binary_crossentropy` (comumente usada quando os pixels estão entre 0 e 1, interpretados como probabilidades) usando o otimizador `adam`.

In [ ]:
autoencoder.compile(optimizer='adam', loss='mse')

# Treinamos o modelo por 20 épocas
# Note que x_train é usado tanto como entrada quanto como alvo (x -> x)
history = autoencoder.fit(x_train, x_train,
                epochs=20,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test),
                verbose=1)

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss durante o treinamento')
plt.show()

### Visualização dos Resultados

Vamos comparar as imagens originais com as reconstruções feitas pelo autoencoder.

In [ ]:
encoded_imgs = encoder.predict(x_test)
decoded_imgs = autoencoder.predict(x_test)

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # Exibir original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n // 2:
        ax.set_title("Originais")

    # Exibir reconstrução
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n // 2:
        ax.set_title("Reconstruções")
plt.show()

### Visualização do Espaço Latente com t-SNE

O "código" aprendido pelo encoder tem dimensão 32. Podemos usar o t-SNE (visto em notebooks anteriores) para visualizar como os dígitos estão organizados nesse espaço latente.

In [ ]:
from sklearn.manifold import TSNE

n_samples = 1000
encoded_subset = np.asarray(encoded_imgs[:n_samples])

(_, _), (_, y_test) = mnist.load_data()
y_test_subset = y_test[:n_samples].astype(int)

tsne = TSNE(n_components=2, random_state=42, init="pca", learning_rate="auto")
tsne_results = tsne.fit_transform(encoded_subset)

plt.figure(figsize=(10, 8))
sc = plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=y_test_subset, cmap="tab10", s=12, alpha=0.7)
plt.colorbar(sc, ticks=range(10))
plt.title("t-SNE do Espaço Latente (32D -> 2D)")
plt.show()

## Denoising Autoencoder

Um Denoising Autoencoder (DAE) é treinado para reconstruir uma entrada limpa a partir de uma versão corrompida (ruidosa) dela. Isso força o modelo a aprender características robustas e a ignorar o ruído.

### Adicionando Ruído

Vamos adicionar ruído gaussiano às imagens do MNIST.

In [ ]:
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

# Clipar os valores para ficar entre 0 e 1
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

# Visualizar imagens ruidosas
n = 10
plt.figure(figsize=(20, 2))
for i in range(n):
    ax = plt.subplot(1, n, i + 1)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.suptitle("Entrada Ruidosa", fontsize=16)
plt.show()

### Modelo DAE

Usaremos uma arquitetura profunda (Deep Autoencoder) para melhor capacidade.

In [ ]:
input_img = Input(shape=(784,))

# Encoder
x = Dense(128, activation='relu')(input_img)
x = Dense(64, activation='relu')(x)
encoded = Dense(32, activation='relu')(x)

# Decoder
x = Dense(64, activation='relu')(encoded)
x = Dense(128, activation='relu')(x)
decoded = Dense(784, activation='sigmoid')(x)

dae = Model(input_img, decoded)
dae.compile(optimizer='adam', loss='mse')

In [ ]:
# O treino agora usa x_train_noisy como entrada, mas x_train (limpo) como alvo
dae.fit(x_train_noisy, x_train,
        epochs=20,
        batch_size=128,
        shuffle=True,
        validation_data=(x_test_noisy, x_test),
        verbose=1)

### Resultados do Denoising


In [ ]:
decoded_imgs = dae.predict(x_test_noisy)

n = 10
plt.figure(figsize=(20, 6))
for i in range(n):
    # Ruidoso
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n // 2: ax.set_title("Ruidoso")

    # Reconstruído
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n // 2: ax.set_title("Reconstruído (Limpo)")

    # Original (Ground Truth)
    ax = plt.subplot(3, n, i + 1 + 2*n)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n // 2: ax.set_title("Original")

plt.show()

## Exercícios

### Exercício 1

Altere o `encoding_dim` no Autoencoder Simples para um valor muito pequeno (ex: 4 ou 2) e observe a qualidade da reconstrução. O que acontece com os dígitos? Eles ficam borrados? Ainda são reconhecíveis?

In [ ]:
# Exercício 1: Seu código aqui


### Exercício 2

O Autoencoder Simples usou apenas uma camada oculta. Tente adicionar mais camadas (como fizemos no Denoising AE) mas para a tarefa de reconstrução simples (sem ruído). Compare a perda final (loss) e a nitidez das imagens com o modelo de camada única.

In [ ]:
# Exercício 2: Seu código aqui
